# Prosper Exploratory Analysis
## by Mabelly Tuchsznajder

## Preliminary Wrangling

Prosper is a financial company that matches investors and borrowers in a financially and socially rewarding way.

The goal of this data analysis project is to understand the factors that are related to loans that were charged off or defaulted. 

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

#### High-level overview of Prosper dataset

In [2]:
loan_df = pd.read_csv('raw/prosperLoanData.csv')
loan_df.head()

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20


In [3]:
loan_df.shape

(113937, 81)

In [4]:
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
ListingKey                             113937 non-null object
ListingNumber                          113937 non-null int64
ListingCreationDate                    113937 non-null object
CreditGrade                            28953 non-null object
Term                                   113937 non-null int64
LoanStatus                             113937 non-null object
ClosedDate                             55089 non-null object
BorrowerAPR                            113912 non-null float64
BorrowerRate                           113937 non-null float64
LenderYield                            113937 non-null float64
EstimatedEffectiveYield                84853 non-null float64
EstimatedLoss                          84853 non-null float64
EstimatedReturn                        84853 non-null float64
ProsperRating (numeric)                84853 non-null float64
ProsperRating (Alpha) 

#### Data Cleaning Process

In [7]:
# Define features of interest for this analysis
features = ['ProsperScore', 'ListingCategory (numeric)', 'EmploymentStatus', 'EmploymentStatusDuration', \
            'IsBorrowerHomeowner', 'CurrentCreditLines', 'OpenRevolvingAccounts', 'DebtToIncomeRatio', \
            'IncomeVerifiable', 'Recommendations', 'LoanStatus']

In [8]:
# Look at the dataset
loan_df[features].sample(5)

,ProsperScore,ListingCategory (numeric),EmploymentStatus,EmploymentStatusDuration,IsBorrowerHomeowner,CurrentCreditLines,OpenRevolvingAccounts,DebtToIncomeRatio,IncomeVerifiable,Recommendations,LoanStatus
35265,5.0,13,Employed,69.0,False,13.0,5,0.34,True,0,Past Due (>120 days)
36360,11.0,2,Employed,22.0,True,9.0,4,0.13,True,0,Current
72621,5.0,7,Employed,16.0,False,9.0,6,0.18,True,0,Current
19175,4.0,7,Employed,161.0,True,5.0,4,0.05,True,0,Completed
104994,4.0,1,Employed,126.0,True,14.0,10,0.24,True,0,Current


In [9]:
loan_df[features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 11 columns):
ProsperScore                 84853 non-null float64
ListingCategory (numeric)    113937 non-null int64
EmploymentStatus             111682 non-null object
EmploymentStatusDuration     106312 non-null float64
IsBorrowerHomeowner          113937 non-null bool
CurrentCreditLines           106333 non-null float64
OpenRevolvingAccounts        113937 non-null int64
DebtToIncomeRatio            105383 non-null float64
IncomeVerifiable             113937 non-null bool
Recommendations              113937 non-null int64
LoanStatus                   113937 non-null object
dtypes: bool(2), float64(4), int64(3), object(2)
memory usage: 8.0+ MB


In [10]:
# Check unique LoanStatus
loan_df['LoanStatus'].unique()

array(['Completed', 'Current', 'Past Due (1-15 days)', 'Defaulted',
       'Chargedoff', 'Past Due (16-30 days)', 'Cancelled',
       'Past Due (61-90 days)', 'Past Due (31-60 days)',
       'Past Due (91-120 days)', 'FinalPaymentInProgress',
       'Past Due (>120 days)'], dtype=object)

In [13]:
# Define LoanStatus as a category
loan_status_dict = {'LoanStatus': ['Defaulted','Chargedoff','Past Due (>120 days)',
                                   'Past Due (91-120 days)', 'Past Due (61-90 days)', 
                                   'Past Due (31-60 days)', 'Past Due (16-30 days)', 'Past Due (1-15 days)', 
                                   'Current', 'FinalPaymentInProgress', 'Completed']}

ordered_loan_status = pd.api.types.CategoricalDtype(ordered = True, categories = loan_status_dict['LoanStatus'])

loan_df['LoanStatus'] = loan_df['LoanStatus'].astype(ordered_loan_status)

In [11]:
# Check unique Employment Status
loan_df['EmploymentStatus'].unique()

array(['Self-employed', 'Employed', 'Not available', 'Full-time', 'Other',
       nan, 'Not employed', 'Part-time', 'Retired'], dtype=object)

In [16]:
# Fill nan values with 'Not available'
loan_df['EmploymentStatus'].fillna(value='Not available', inplace=True)

In [18]:
# Define EmploymentStatus as a category
employment_status_dict = {'EmploymentStatus': ['Not available', 'Other', 'Not employed', 'Self-employed', 
                                               'Part-time', 'Full-time', 'Employed', 'Retired']}

ordered_employment_status = pd.api.types.CategoricalDtype(ordered = True, categories = employment_status_dict['EmploymentStatus'])

loan_df['EmploymentStatus'] = loan_df['EmploymentStatus'].astype(ordered_employment_status)

In [19]:
# Decode ListingCategory (numeric) feature
list_dict = {0 : 'Not Available', 1 : 'Debt Consolidation', 2 : 'Home Improvement', 3: 'Business', 
             4 : 'Personal Loan', 5 : 'Student Use', 6 : 'Auto', 7 : 'Other', 8 : 'Baby&Adoption', 
             9 : 'Boat', 10 : 'Cosmetic Procedure', 11 : 'Engagement Ring', 12 : 'Green Loans',
             13 : 'Household Expenses', 14 : 'Large Purchases', 15 : 'Medical/Dental', 16 : 'Motorcycle',
             17 : 'RV', 18 : 'Taxes', 19 : 'Vacation', 20 : 'Wedding Loans'}

loan_df['ListingCategory'] = loan_df['ListingCategory (numeric)'].map(list_dict)

In [20]:
# Look at the new ListingCategory feature
loan_df[['ListingCategory (numeric)', 'ListingCategory']].sample(10)

,ListingCategory (numeric),ListingCategory
35787,0,Not Available
88368,0,Not Available
6951,6,Auto
13469,15,Medical/Dental
19068,3,Business
52749,1,Debt Consolidation
31220,7,Other
81449,1,Debt Consolidation
32297,1,Debt Consolidation
34825,4,Personal Loan


### What is the structure of your dataset?

There are 113,937 loans in the dataset with 81 features. This analysis will focus on factors that are related to Prosper loans status (`LoanStatus`).

The independent variables of interest are:
- `ProsperScore`
- `ListingCategory`
- `EmploymentStatus`
- `EmploymentStatusDuration`
- `IsBorrowerHomeowner`
- `CurrentCreditLines`
- `OpenRevolvingAccounts`
- `DebtToIncomeRatio`
- `IncomeVerifiable`
- `Recommendations`


### What is/are the main feature(s) of interest in your dataset?

I'm most interested in figuring out what features are best for predicting the loan status in the dataset.

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?
I expect that the ListingCategory variable, as well as all borrower features listed above will have the strongest effect on each LoanStatus. 


## Univariate Exploration

> In this section, investigate distributions of individual variables. If
you see unusual points or outliers, take a deeper look to clean things up
and prepare yourself to look at relationships between variables.

> Make sure that, after every plot or related series of plots, that you
include a Markdown cell with comments about what you observed, and what
you plan on investigating next.

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!